# Importing Libraries

In [1]:
#Imports
import os
import pandas as pd
import numpy as np
import os
from google.colab import files

# Cloning GitHub Repository

In [2]:
!git clone https://github.com/ShafaqFatimaMughal/Kaavish-Mai-Transformer-Based-Urdu-Chatbot.git
path = "/content/Kaavish-Mai-Transformer-Based-Urdu-Chatbot/Datasets/Initial"

Cloning into 'Kaavish-Mai-Transformer-Based-Urdu-Chatbot'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 56 (delta 20), reused 39 (delta 6), pack-reused 0
Unpacking objects: 100% (56/56), done.


# Extracting Data

In [3]:
self_path = path + "/Self-Curated"
data = {}

for root, dirs, filess in os.walk(self_path): # We need the files in this path / diretory

    for index, name in enumerate(filess): 
        naming_index = name.split("/")[-1].split(" ")[1] # We want just the filename
        filename = os.path.join(root, name)
        df = pd.read_csv(filename, usecols = ['Intent', 'Response', 'Relevance'])
        df = df[df['Relevance'] != 0]
        data[naming_index] = df # Save df in dict
        print(filename) 

dataset = pd.DataFrame(columns=['Intent', 'Response', "Relevance"])

# Concatenating dataset together
for i in data.keys(): 
  dataset = dataset.append(data[i])
dataset.info

/content/Kaavish-Mai-Transformer-Based-Urdu-Chatbot/Datasets/Initial/Self-Curated/Kaavish D5 - Hello Clue.csv
/content/Kaavish-Mai-Transformer-Based-Urdu-Chatbot/Datasets/Initial/Self-Curated/Kaavish D0 - General Mai.csv
/content/Kaavish-Mai-Transformer-Based-Urdu-Chatbot/Datasets/Initial/Self-Curated/Kaavish D2 - Mestrual Hygeine.csv
/content/Kaavish-Mai-Transformer-Based-Urdu-Chatbot/Datasets/Initial/Self-Curated/Kaavish D6 - Flo.csv
/content/Kaavish-Mai-Transformer-Based-Urdu-Chatbot/Datasets/Initial/Self-Curated/Kaavish D3 - Sexually Transmitted Infections.csv
/content/Kaavish-Mai-Transformer-Based-Urdu-Chatbot/Datasets/Initial/Self-Curated/Kaavish D4 - New Periods Curriculum.csv
/content/Kaavish-Mai-Transformer-Based-Urdu-Chatbot/Datasets/Initial/Self-Curated/Kaavish D1 - Fertility Dost Chatbox_2.csv


<bound method DataFrame.info of                                                Intent  \
0                                              Hello    
1   I had some questions about my periods, will yo...   
2                  I wanted to know what an ovary is?   
3              I was wondering what uterus looks like   
4                  What happens in a menstrual cycle    
..                                                ...   
28                                      What is Yoga?   
29                                  What is Ayurveda?   
30                    What disturbs Female Fertility?   
31  I haven’t yet visited the doctor to test my fe...   
32                                                NaN   

                                             Response  Relevance  
0    Hello! My name is Mai, how can I help you today?        NaN  
1   Sure, in fact you have reached the right perso...        NaN  
2   The ovaries are small, oval-shaped glands loca...        1.0  
3   The uterus 

# Cleaning Data

In [4]:
dataset.drop(columns=["Relevance"], inplace=True)
dataset.dropna(inplace=True)
dataset.drop_duplicates(inplace=True)
dataset.info

<bound method DataFrame.info of                                                Intent  \
0                                              Hello    
1   I had some questions about my periods, will yo...   
2                  I wanted to know what an ovary is?   
3              I was wondering what uterus looks like   
4                  What happens in a menstrual cycle    
..                                                ...   
27                                What is Homeopathy?   
28                                      What is Yoga?   
29                                  What is Ayurveda?   
30                    What disturbs Female Fertility?   
31  I haven’t yet visited the doctor to test my fe...   

                                             Response  
0    Hello! My name is Mai, how can I help you today?  
1   Sure, in fact you have reached the right perso...  
2   The ovaries are small, oval-shaped glands loca...  
3   The uterus is a hollow muscular organ located ...  
4  

In [5]:
# Reformatting dataframe data to new dataframe
final_data = pd.DataFrame(columns=["Name", "Line"])
for i, row in dataset.iterrows():
  user = row["Intent"] 
  response = row["Response"]
  final_data = final_data.append({"Name": "User", "Line": user}, ignore_index=True) 
  final_data = final_data.append({"Name": "Mai", "Line": response}, ignore_index=True)

In [6]:
contexted = []
n = 7 # Context length 

for i in range(n, len(final_data['Line'])):
  row = []
  prev = i - 1 - n # We additionally substract 1, so row will contain current responce and 7 previous responces  
  for j in range(i, prev, -1):
    row.append(final_data['Line'][j])
  contexted.append(row)  

In [7]:
len(contexted)

721

In [8]:
# Making list of columns according to context len
columns = ['response', 'context'] 
columns = columns + ['context/'+str(i) for i in range(n-1)]
columns

['response',
 'context',
 'context/0',
 'context/1',
 'context/2',
 'context/3',
 'context/4',
 'context/5']

In [9]:
df = pd.DataFrame.from_records(contexted, columns=columns) # Make dataframe
df.head(5)

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
0,The uterus is a hollow muscular organ located ...,I was wondering what uterus looks like,"The ovaries are small, oval-shaped glands loca...",I wanted to know what an ovary is?,"Sure, in fact you have reached the right perso...","I had some questions about my periods, will yo...","Hello! My name is Mai, how can I help you today?",Hello
1,What happens in a menstrual cycle,The uterus is a hollow muscular organ located ...,I was wondering what uterus looks like,"The ovaries are small, oval-shaped glands loca...",I wanted to know what an ovary is?,"Sure, in fact you have reached the right perso...","I had some questions about my periods, will yo...","Hello! My name is Mai, how can I help you today?"
2,A menstrual cycle starts on the first day of t...,What happens in a menstrual cycle,The uterus is a hollow muscular organ located ...,I was wondering what uterus looks like,"The ovaries are small, oval-shaped glands loca...",I wanted to know what an ovary is?,"Sure, in fact you have reached the right perso...","I had some questions about my periods, will yo..."
3,I am not sure if I understood what a uterine c...,A menstrual cycle starts on the first day of t...,What happens in a menstrual cycle,The uterus is a hollow muscular organ located ...,I was wondering what uterus looks like,"The ovaries are small, oval-shaped glands loca...",I wanted to know what an ovary is?,"Sure, in fact you have reached the right perso..."
4,"Before Ovulation, this cycle physically change...",I am not sure if I understood what a uterine c...,A menstrual cycle starts on the first day of t...,What happens in a menstrual cycle,The uterus is a hollow muscular organ located ...,I was wondering what uterus looks like,"The ovaries are small, oval-shaped glands loca...",I wanted to know what an ovary is?


# Further Cleaning 
## Removing some identifiers and explicit text

In [10]:
# !pip install clean-text
# !pip install better-profanity
# !pip install tqdm

# # Imports
# from cleantext import clean
# from tqdm import tqdm

In [11]:
# # Getting columns for the new data frame
# columns = df.columns
# new_df = pd.DataFrame(columns=columns)
# # Iterating over dataframe
# for index, row in tqdm(df.iterrows()):
#   # Creating new data to append to new dataframe
#   row_data = {}
#   # Iterating over all columns of a row and cleaning the data to add to the dict
#   for col in columns:
#     row_data[col] = clean(row[col], no_emoji=True, lower=True, no_punct = True)
#   # Appending the data to the new dataframe
#   new_df = new_df.append(row_data, ignore_index=True)

# new_df.to_csv('self_curated.csv', index=False)
# files.download('self_curated.csv')
# print(new_df.info())

# new_df

# Saving and Downloading

In [12]:
df.to_csv('self_curated.csv', index=False)
files.download('self_curated.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>